In [ ]:
#we are going to import the geracao dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
dataset = pd.read_csv('geracao.csv')
del dataset["Unnamed: 0"]
print(dataset.head)

In [ ]:
#we are going to plot every single data in the dataset to see how it looks like
dataset.plot(subplots=True)
plt.show()
#saving the plot as a image

Na célula abaixo estamos limpando os dados, excluimos dados que tinham a condutividade térmica acima de 500, área acima de 30 e dt/dx maior que -50

In [ ]:
#vamos podar o dataset removendo instancias em que o dt/dx é muito grande, maior que -50
before = dataset.shape[0]

dataset = dataset[dataset.iloc[:,4] > -30]
dataset = dataset[dataset.iloc[:,3] < 300]
dataset = dataset[dataset.iloc[:,2] < 30]
#quantidades de rows que temos no dataset
after = dataset.shape[0]
print("antes: ", before)
print("depois: ", after)
print("perda: ", (before - after))


In [ ]:
dataset.plot(subplots=True)
plt.show()


In [ ]:
dados = dataset[["tx_transf", "area","condutividade","dtdx"]]
target = dataset["flx_calor"]

In [ ]:
X_train_full, x_test, y_train_full, y_test = train_test_split(dados, target, test_size=0.25, random_state=42)


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
x_test = scaler.transform(x_test)


In [ ]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3,activation="relu", input_shape=[4]):
    model = keras.models.Sequential()
    options = {"input_shape": input_shape}
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation=activation, **options))
        options = {}
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [ ]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(10, 256),
    "learning_rate": reciprocal(3e-5, 3e-3),
    "activation": ["tanh"],
}

In [ ]:
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=5,n_jobs=1,cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=50,
                    validation_data=(X_valid, y_valid),
                    callbacks=[keras.callbacks.EarlyStopping(patience=5,monitor="val_loss")])

In [ ]:
hist = rnd_search_cv.best_estimator_.model.history

In [ ]:
rnd_search_cv.best_params_

In [ ]:
rnd_search_cv.best_score_

In [ ]:
model = rnd_search_cv.best_estimator_.model

In [ ]:
model.summary()

In [ ]:
print(hist)

In [ ]:
#ploting the loss and the validation loss
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()


In [ ]:
#average of the loss and the val loss
#weighted average
loss = hist["loss"]
val_loss = hist["val_loss"]
average_loss = np.average(loss)
average_val_loss = np.average(val_loss)
print("average loss: ", average_loss)
print("average val loss: ", average_val_loss)


In [ ]:
mse_test = model.evaluate(x_test, y_test)

In [ ]:
#ploting the predictions and the actual values
y_pred = model.predict(x_test)
y_pred = y_pred.flatten()
#ploting the predictions and the actual values

In [ ]:

plt.plot(y_pred, color="red", label="pred")
plt.plot(y_test.values, color="green", label="actual")
plt.legend(loc="upper left")
plt.show()

In [ ]:
#making a scatter graph of the real values and a line of the predictions
plt.scatter(y_test.values[:1000], y_pred[:1000], color="red")
plt.plot([0, 1], [0, 1], transform=plt.gca().transAxes)
plt.show()


In [ ]:
#plotting just the first 100 predictions and values
plt.plot(y_pred[:100], color="red", label="pred")
plt.plot(y_test.values[:100], color="green", label="actual")
plt.legend(loc="upper left")
plt.show()


In [ ]:
#predicting the first 10 values
for i in range(10):
    print("pred: ", y_pred[i], "actual: ", y_test.values[i])


In [ ]:
#we are going to check if the loss of the model that we trained now is lower than the model that was saved in the file
#we are going to load the model that was saved in the file
old_model = keras.models.load_model("model.h5")
old_loss = old_model.evaluate(x_test, y_test)
#checking if the loss of the model that we trained now is lower than the model that was saved in the file
if mse_test < old_loss:
    model.save("model.h5")
    print("saved")

In [ ]:
#importing the dataset test.csv
dataset_test = pd.read_csv('teste.csv')
#setting up the target and the data
target = dataset_test["flx_calor"]
dados = dataset_test[["tx_transf", "area","condutividade","dtdx"]]
#scaling the data
dados = scaler.transform(dados)

In [ ]:
#showing the first 10 predctions and the real values
y_pred = model.predict(dados)
y_pred = y_pred.flatten()
for i in range(10):
    print("pred: ", y_pred[i], "actual: ", target.values[i])

In [ ]:
#Plotting the predictions and the real values
plt.plot(y_pred, color="red", label="pred")
plt.plot(target.values, color="green", label="actual")
plt.legend(loc="upper left")
plt.show()

